In [1]:
from random import seed, random, randrange
from csv import reader
from math import exp
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import numpy as np
import csv
import pandas as pd

In [2]:
filename = 'c:/Users/vijay/Downloads/data.csv'

In [3]:
import pandas as pd
import numpy as np
import csv
from random import seed, random, randrange
from math import exp

# Load a CSV file
def loadCsv(filename):
    trainSet = []
    with open(filename, 'r') as file:
        lines = csv.reader(file)
        dataset = list(lines)
        for i in range(len(dataset)):
            for j in range(len(dataset[i])):
                # Remove commas from numbers
                dataset[i][j] = float(dataset[i][j].replace(',', ''))
            trainSet.append(dataset[i])
    return trainSet

# Min-max normalization
def normalize(dataset):
    min_vals = np.min(dataset, axis=0)
    max_vals = np.max(dataset, axis=0)
    for row in dataset:
        row[:-1] = (row[:-1] - min_vals[:-1]) / (max_vals[:-1] - min_vals[:-1])

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = []
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation

# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

# Calculate the derivative of a neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network) - 1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate, mu):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                temp = l_rate * neuron['delta'] * inputs[j] + mu * neuron['prev'][j]
                neuron['weights'][j] += temp
                neuron['prev'][j] = temp
            temp = l_rate * neuron['delta'] + mu * neuron['prev'][-1]
            neuron['weights'][-1] += temp
            neuron['prev'][-1] = temp

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs, mu):
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[int(row[-1])] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate, mu)

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = []
    hidden_layer_1 = [{'weights': [random() for i in range(n_inputs + 1)], 'prev': [0 for i in range(n_inputs + 1)]}
                      for i in range(n_hidden)]
    network.append(hidden_layer_1)
    hidden_layer_2 = [{'weights': [random() for i in range(n_hidden + 1)], 'prev': [0 for i in range(n_hidden + 1)]}
                      for i in range(n_hidden)]
    network.append(hidden_layer_2)
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)], 'prev': [0 for i in range(n_hidden + 1)]}
                    for i in range(n_outputs)]
    network.append(output_layer)
    return network

# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden, mu, n_outputs):
    n_inputs = len(train[0]) - 1
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs, mu)
    predictions = []
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return predictions

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross-validation split
def run_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = []
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = []
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Test the backpropagation algorithm on the vowel dataset

seed(1)
data = pd.read_csv("C:/Users/vijay/Downloads/data.csv")
dataset = data.values.tolist()

# Convert class labels to integers
class_labels = sorted(set([row[-1] for row in dataset]))
n_class = len(class_labels)
for row in dataset:
    row[-1] = class_labels.index(row[-1])

# Normalize the dataset
normalize(dataset)

# Parameters for the algorithm
learning_rate = 0.3
epochs = 500
hidden_neurons = 4
momentum = 0.9
folds = 5

# Run the backpropagation algorithm with cross-validation
scores = run_algorithm(dataset, back_propagation, folds, learning_rate, epochs, hidden_neurons, momentum, n_class)

# Print the average accuracy and other metrics
print('Scores:', scores)
print('Mean Accuracy: %.2f%%' % (sum(scores) / float(len(scores))))


Scores: [82.75862068965517, 81.03448275862068, 78.16091954022988, 78.73563218390804, 84.48275862068965]
Mean Accuracy: 81.03%


In [4]:
import pandas as pd
import numpy as np
import csv
from random import seed, random, randrange
from math import exp

# ... (The rest of the code remains the same as before)

# Function to write output to a CSV file
def write_output_to_csv(output_dict, filename):
    df = pd.DataFrame(output_dict)
    df.to_csv(filename, index=False)

# Test the backpropagation algorithm on the vowel dataset
# ... (The rest of the code remains the same as before)

# Create a copy of the dataset
dataset_copy = dataset.copy()

# Create a dictionary to store the output
output_dict = {
    'Fold': list(range(1, folds+1)),
    'Accuracy': scores,
    # Include other evaluation metrics in the dictionary
    # ... (e.g., 'Specificity', 'Sensitivity', 'Precision', 'Recall', 'FScore', 'CohenKappa')
}

# Write the output to a CSV file
write_output_to_csv(output_dict, 'output.csv')
